## MPAR notebook

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from math import log, factorial
from tqdm import tqdm
import matplotlib as mpl
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = 10,8
plt.rcParams['font.size'] = 35
plt.style.use(['ggplot','fivethirtyeight'])
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from numba import njit
import tensorflow as tf
from scipy.stats import wilcoxon

In [2]:
os.chdir('../')

In [3]:
data_type= '15min' # 'hourly'

data_list = ['EURUSD','GBPUSD','USDCHF','USDJPY']
data_list = [f'data/processed/{x}_{data_type}_15min.csv' for x in data_list]

In [4]:
def add_ma(data: pd.DataFrame, period_short: int, period_long: int, d_prev: int, ema:bool=False):
    if ema:
        data['ma_short'] = data['bid'].ewm(span=period_short).mean()
        data['ma_long'] = data['bid'].ewm(span=period_long).mean()
    else:
        data['ma_short'] = data['bid'].rolling(window=period_short).mean()
        data['ma_long'] = data['bid'].rolling(window=period_long).mean()
    data['diff'] = data['ma_short'] - data['ma_long']
    for i in range(1, d_prev+1):
        data[f'diff_{i}'] = data['diff'].shift(i)
    return data.dropna(inplace=False)


In [5]:
%load_ext autoreload
%autoreload
from src.rml_model import Encoder, Decoder, Agent

SEED:  0


In [6]:
from scipy.optimize import minimize

model_config_e = [   
    {'type': 'dense', 'units': 5, 'activation': 'silu'},
]
model_config_d = [   
    {'type': 'dense', 'units': 5, 'activation': 'relu', 'dropout': 0.2},
    {'type': 'dense', 'units': 3, 'activation': 'tanh'},
]
encoder = Encoder(2, model_config_e)
decoder = Decoder(encoder.output.shape[1], model_config_d)
agent = Agent(encoder, decoder)


def ma(x, revert, threshold):
    revert = np.sign(revert)
    return revert*(1 if x > threshold else (-1 if x < -threshold else 0))

# Define the objective function to minimize (negative Sharpe Ratio)
def objective(params, X_train, BA_train, agent):
    threshold, revert = params
    dec_ma = X_train['diff'].map(lambda x: ma(x, threshold=threshold, revert=revert))
    sr_train = agent.utility_function([BA_train.values, dec_ma.values]).numpy()
    return sr_train


def optimize_ma(X_train, BA_train, agent=agent):
    best_objective_value = float('-inf')
    best_params = None

    # Define the range of values to test
    thresholds = np.linspace(0.0001, 1.5, 50)  # 1500 points between 0.0001 and 1.5
    reverts = [-1, 1]

    # Iterate over all combinations of parameters
    for threshold in thresholds:
        for revert in reverts:
            params = (threshold, revert)
            objective_value = objective(params, X_train, BA_train, agent)

            # Check if this is the best set of parameters found so far
            if objective_value > best_objective_value:
                best_objective_value = objective_value
                best_params = params
    return best_params


2025-08-06 17:58:51.298469: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-08-06 17:58:51.298502: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tomek-lab
2025-08-06 17:58:51.298510: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tomek-lab
2025-08-06 17:58:51.298687: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 555.58.2
2025-08-06 17:58:51.298710: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 555.58.2
2025-08-06 17:58:51.298716: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 555.58.2
2025-08-06 17:58:51.298963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions i

In [7]:
sl  = {
'EURUSD':{'short': 58, 'long': 222, 'ma_train': -0.004199},
'GBPUSD':{'short': 57, 'long': 201, 'ma_train': -0.004166},
'USDCHF':{'short': 44, 'long': 225, 'ma_train': -0.002749},
'USDJPY':{'short': 49, 'long': 180, 'ma_train': -0.005136}}

In [9]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import matplotlib as mpl

mpl.rcParams.update({
    # Fonts
    "font.family": "serif",
    "font.size": 18,
    "axes.titlesize": 20,
    "axes.labelsize": 18,

    # Grid and Axes
    "axes.grid": True,
    "grid.alpha": 0.6,
    "grid.linestyle": "--",
    "axes.spines.top": False,
    "axes.spines.right": False,

    # Line and Marker Styles
    "lines.linewidth": 2.5,          # Thicker lines for better visibility
    "lines.markersize": 8,           # Larger markers

    # Tick Parameters
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "xtick.direction": "out",
    "ytick.direction": "out",

    # Legend
    "legend.fontsize": 14,           # Larger legend font
    "legend.frameon": True,          # Add subtle frame for separation
    "legend.framealpha": 0.8,        # Slightly transparent frame
    "legend.loc": 'best',            # Automatic best placement

    # Figure Layout
    "figure.dpi": 300,
    "savefig.dpi": 600,              # Increase for clearer saved images
    "figure.figsize": [7.5, 4],      # Slightly larger figures
})

import seaborn as sns
colors = sns.color_palette("colorblind")

In [12]:
catalogue = 'results'  
data_type = '15min'

DF = pd.DataFrame()
test_ma = {}
train_ma = {}
for k,v in sl.items():
    name = f"data/processed/{k}_{data_type}.csv"
    data = pd.read_csv(name)

    s = v['short']
    l = v['long']

    df = add_ma(data.copy(), s, l, 0)
    X = df[[col for col in df.columns if 'diff' in col]]
    BA = df[['bid', 'ask']]

    # Split into 90% training and 10% testing.
    # Ensure 'date' column is in datetime format
    data['time'] = pd.to_datetime(data['time'])
    df['time'] = data.loc[df.index, 'time']

    # Split data
    n = int(len(df) * 0.8)
    idx_train = df.index[:n]
    idx_test = df.index[n:]

    X_train, X_test = X.loc[idx_train], X.loc[idx_test]
    BA_train, BA_test = BA.loc[idx_train], BA.loc[idx_test]

    threshold, revert = optimize_ma(X_train, BA_train)
    
    dec_ma_train = X_train['diff'].map(lambda x: ma(x, threshold=threshold, revert=revert))
    sr_ma_train = agent.utility_function([BA_train.values, dec_ma_train.values]).numpy()
    

    train_ma[k.replace('RS_','')] = sr_ma_train

    dec_ma_test = X_test['diff'].map(lambda x: ma(x, threshold=threshold, revert=revert))
    sr_ma_test = agent.utility_function([BA_test.values, dec_ma_test.values]).numpy()
    
    test_ma[k.replace('RS_','')] = sr_ma_test



    res = [x.split('_')[-1] for x in os.listdir(f'results_{k}')]
    res = [float(x)/100 for x in res if '.' in x]

    sure=0.0
    dfs = []
    dir_name = f'{catalogue}_{k}'

    if not os.path.exists(dir_name):
        continue


    for dir in os.listdir(dir_name):
        dir = os.path.join(dir_name, dir)
        temp_df = pd.read_csv(f'{dir}/df.csv')
        temp_df['epoch'] = temp_df.index
        temp_df['currency'] = dir.replace('results_', '')
        dfs.append(temp_df)
    res = pd.concat(dfs, ignore_index=True)

    DF = pd.concat([DF, res], ignore_index=True)

    res = res[res['var_train'] > sure][res.epoch>100][res.srs_train<0.035].groupby('currency').tail(1).srs.values.tolist()

DF[['currency','iter']] = DF.currency.replace('RS_','').str.split('/').values.tolist()
DF['currency'] = DF.currency.str.split('_').str[-1]
DF['sr_ma'] = DF.currency.map(train_ma)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import warnings
from scipy.stats import wilcoxon
from matplotlib.lines import Line2D
import os
import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

# Matplotlib settings for publication visibility
mpl.rcParams.update({
    "font.family": "serif",
    "font.size": 20,
    "axes.titlesize": 22,
    "axes.labelsize": 20,
    "axes.grid": True,
    "grid.alpha": 0.7,
    "grid.linestyle": "--",
    "axes.spines.top": False,
    "axes.spines.right": False,
    "lines.linewidth": 3,
    "lines.markersize": 10,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "legend.fontsize": 18,
    "legend.frameon": True,
    "legend.framealpha": 1.0,
    "legend.edgecolor": "black",
    "figure.dpi": 300,
    "savefig.dpi": 800,
    "figure.figsize": [8, 5],
})

modern_palette = sns.color_palette("Set2", 4)
highlight_color = "#e74c3c"

sure = 0.0
sr_train_max = 0.035

# 1. Final Performance Chart (saved as draft_performance_final.eps)
for final in [True]:
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    axs = axs.ravel()

    print("Final Performance" if final else "Best checkpoint Performance")

    for idx, (k, v) in enumerate(sl.items()):
        res = DF[DF.currency == k]
        res_max = (
            res[(res['var_train'] > sure) &
                (res['epoch'] > 50) & 
                (res['epoch'] < 200) & 
                (res['srs_train'] >= 0.0) & 
                (res['srs_train'] < sr_train_max)]
            .groupby('iter').apply(lambda x: x.tail(1) if final else x.max()).srs.values.tolist()
        )

        if len(res_max) > 0:
            stat, p = wilcoxon(res_max, [test_ma[k]] * len(res_max), alternative='two-sided', zero_method='wilcox')
            print(f"{k}: mean(res_max)={np.mean(res_max):.4f}, max(res_max)={max(res_max):.4f}, test_ma={test_ma[k]:.4f}, Wilcoxon p-value={p:.4f}")
        else:
            print(f"{k}: No data for Wilcoxon test.")

        axs[idx].hist(res_max, bins=20, color=modern_palette[idx], alpha=0.9, range=(-0.018, 0.015))
        axs[idx].axvline(test_ma[k], color=highlight_color, linestyle='--', linewidth=3, label="MACD" if idx == 0 else None)
        axs[idx].set_title(f'{k}', fontweight='bold', pad=12)
        axs[idx].set_xlabel('Sharpe Ratio')
        axs[idx].set_ylabel('Count')
        axs[idx].set_xlim(-.025, 0.025)
        if not idx:
            axs[idx].legend(loc='upper left', fontsize=18, facecolor='white', framealpha=1.0)
        axs[idx].tick_params(axis='both', which='major', labelsize=16)
        axs[idx].spines['top'].set_visible(False)
        axs[idx].spines['right'].set_visible(False)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig("results_plot/draft_performance_final.eps", format="eps", bbox_inches='tight', facecolor='white')
    plt.show()

# 2. Max Performance Chart (saved as max_performance.eps)
for final in [True]:
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    axs = axs.ravel()

    print("Max Performance")

    for idx, (k, v) in enumerate(sl.items()):
        res = DF[DF.currency == k]
        res_max = (
            res[(res['var_train'] > sure) &
                (res['epoch'] < 75) & 
                (res['srs_train'] >= 0.0) & 
                (res['srs_train'] < sr_train_max)]
            .groupby('iter').apply(lambda x: x.tail(1) if final else x.max()).srs.values.tolist()
        )

        if len(res_max) > 0:
            stat, p = wilcoxon(res_max, [test_ma[k]] * len(res_max), alternative='two-sided', zero_method='wilcox')
            print(f"{k}: mean(res_max)={np.mean(res_max):.4f}, max(res_max)={max(res_max):.4f}, test_ma={test_ma[k]:.4f}, Wilcoxon p-value={p:.4f}")
        else:
            print(f"{k}: No data for Wilcoxon test.")

        axs[idx].hist(res_max, bins=20, color=modern_palette[idx], alpha=0.9, range=(-0.018, 0.015))
        axs[idx].axvline(test_ma[k], color=highlight_color, linestyle='--', linewidth=3, label="MACD" if idx == 0 else None)
        axs[idx].set_title(f'{k}', fontweight='bold', pad=12)
        axs[idx].set_xlabel('Sharpe Ratio')
        axs[idx].set_ylabel('Count')
        axs[idx].set_xlim(-.025, 0.025)
        if not idx:
            axs[idx].legend(loc='upper left', fontsize=18, facecolor='white', framealpha=1.0)
        axs[idx].tick_params(axis='both', which='major', labelsize=16)
        axs[idx].spines['top'].set_visible(False)
        axs[idx].spines['right'].set_visible(False)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig("results_plot/max_performance.eps", format="eps", bbox_inches='tight', facecolor='white')
    plt.show()

# 3. Training Progress Chart (saved as training.eps)
fig, axs = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)
axs = axs.ravel()

for idx, currency in enumerate(DF.currency.unique()):
    tmp = DF[(DF.currency == currency) & (DF.var_train > 0.0) & (DF.epoch < 200)]
    ax = axs[idx]
    for i in tmp.iter.unique():
        tmp2 = tmp[tmp.iter == i]
        ax.plot(tmp2.epoch * 20, tmp2.srs, color=modern_palette[0], alpha=0.3, linewidth=2, label='Test SR (runs)' if i == tmp.iter.unique()[0] else None)
        ax.plot(tmp2.epoch * 20, tmp2.srs_train, color=modern_palette[1], alpha=0.3, linewidth=2, label='Train SR (runs)' if i == tmp.iter.unique()[0] else None)

    avg = tmp.groupby(tmp.epoch // 10).mean(numeric_only=True)
    ax.plot(avg.epoch * 20, avg.srs, color=modern_palette[2], linewidth=3, label='Mean Test SR' if idx == 0 else None, zorder=10)
    ax.plot(avg.epoch * 20, avg.srs_train, color=modern_palette[3], linewidth=3, linestyle='--', label='Mean Train SR' if idx == 0 else None, zorder=10)

    ax.set_ylim(-0.013, 0.015)
    ax.set_title(currency, fontweight='bold', fontsize=20)
    ax.set_xlabel('Episode')
    if idx == 0:
        ax.set_ylabel('Sharpe Ratio')
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.tick_params(axis='both', which='major', labelsize=16)

custom_lines = [
    Line2D([0], [0], color=modern_palette[0], alpha=0.3, lw=3, label='Test SR (runs)'),
    Line2D([0], [0], color=modern_palette[1], alpha=0.3, lw=3, label='Train SR (runs)'),
    Line2D([0], [0], color=modern_palette[2], lw=3, label='Mean Test SR'),
    Line2D([0], [0], color=modern_palette[3], lw=3, linestyle='--', label='Mean Train SR'),
]
fig.legend(custom_lines, [l.get_label() for l in custom_lines], loc='lower center', ncol=4, fontsize=18, frameon=True, facecolor='white', framealpha=1.0, bbox_to_anchor=(0.5, -0.08))

plt.tight_layout()
plt.subplots_adjust(bottom=0.25)
plt.savefig("results_plot/training.eps", format="eps", bbox_inches='tight', facecolor='white')
plt.show()